<a href="https://colab.research.google.com/github/techthumb1/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-make-features/LS_DS_112_Make_Features_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [0]:
#Load the zipped file 
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

In [0]:
#Unzip the data into a csv file
!unzip LoanStats_2018Q4.csv.zip

In [0]:
#Look at the head raw data
!head LoanStats_2018Q4.csv

In [0]:
#Look at the tail
!tail LoanStats_2018Q4.csv

Load Lending Clud Data

In [0]:
#Use the pandas module read and put data into dataframe
import pandas as pd
df = pd.read_csv('LoanStats_2018Q4.csv', skiprows=1, skipfooter=2, engine='python')
df.head()

In [0]:
#What is the shape of my file?
print(df.shape)
print(df.columns)

In [0]:
#Let's look at tehe shape of the dataset
df.shape

In [0]:
#Check the NaN values
pd.options.display.max_rows=999
pd.options.display.max_columns=100
df.isnull().sum().sort_values(ascending=False)

Working with strings

In [0]:
# Let's identify a column that shouldn't be a string.
df['int_rate'].head()

In [0]:
#
df.isnull().sum().sort_values(ascending=False)

In [0]:
## what is the data type of this column?
df['int_rate'].dtype

In [0]:
## what is the value of the first cell in this column?
int_rate0=df.loc[0, 'int_rate']
print(int_rate0)
int_rate0

In [0]:
## we need a python method for removing string values
int_rate0.strip()

In [0]:
## get rid of the percent sign
int_rate0.strip('%')

In [0]:
## get rid of both using "chained methods"
int_rate0.strip().strip('%')

In [0]:
## Finally, we can now convert from string to float
float(int_rate0.strip().strip('%'))

In [0]:
## confirm that did what we wanted
print(type(int_rate0.strip().strip('%')))
print(type(float(int_rate0.strip().strip('%'))))

Conver int_rate

In [0]:
# write your function
def int_rate_2_float(mystring):
  return float(mystring.strip().strip('%'))

In [0]:
## test your function
int_rate_2_float(int_rate0)

In [0]:
## apply your function to the whole columns
df['int_rate_float']=df['int_rate'].apply(int_rate_2_float)
df.columns

In [0]:
## compare the old and new columns
df[['int_rate', 'int_rate_float']].head()

In [0]:
# confirm that we can now employ math on this
df['int_rate_double']=df['int_rate_float']*2
df[['int_rate', 'int_rate_float', 'int_rate_double']].sample(5)

Clean emp_title

In [0]:
# Let's explore this column
df[['emp_title', 'int_rate']].sample(6)

In [0]:
# brackets for columns
df[['emp_title', 'int_rate']].head()

In [0]:
# what are all possible titles?
df['emp_title'].nunique()

In [0]:
# what are all possible titles?
df['emp_title'].nunique()

# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01